In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,f1_score,accuracy_score)
from sklearn.model_selection import cross_val_score

recipes=pd.read_csv("recipes.csv")
recipes

,'acorn squash',adobo,'african birdseye chile pepper',ale,'aleppo pepper','alfalfa sprouts','alfredo sauce',allspice,almond,'almond butter',...,yeast,'yellow curry paste','yellow food coloring','yellow split pea','yellow squash',yogurt,zaatar,zest,zucchini,cuisine
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4232,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4233,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11
4234,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,11


In [44]:
#Randomly select 20% data replaced to be Nan
#Change of data into numpy array
def recipes_Nan_process(recipes,percentages):
    recipes_array=np.array(recipes.drop(['cuisine'],axis=1))

    #reshape our array into 1-d and randomly select 20% of data to be Nan
    recipes_row=recipes_array.shape[0]
    recipes_colunm=recipes_array.shape[1]
    recipes_array=recipes_array.reshape((recipes_row*recipes_colunm))
    recipes_array=recipes_array.astype(float)
    #setting the random seed to make every filling method get the same data set
    random.seed(a=5)
    random_sample=random.sample(range(0,len(recipes_array)),int(len(recipes_array)*percentages))
    
    for i in random_sample:
        recipes_array[i]=np.nan
    recipes_array=recipes_array.reshape((recipes_row,recipes_colunm))
    
    #check whether our nan replacement successed
    #print(recipes_array==np.array(recipes.drop(['cuisine'],axis=1)))

    recipes_nan=pd.DataFrame(recipes_array)
    #add cuisine colunm into recipes_nan
    recipes_nan['cuisine']=recipes['cuisine']
    recipes_nan.columns=recipes.columns
    return(recipes_nan)

recipes_nan=recipes_Nan_process(recipes,0.2)
recipes_nan

,'acorn squash',adobo,'african birdseye chile pepper',ale,'aleppo pepper','alfalfa sprouts','alfredo sauce',allspice,almond,'almond butter',...,yeast,'yellow curry paste','yellow food coloring','yellow split pea','yellow squash',yogurt,zaatar,zest,zucchini,cuisine
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0
3,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0
4,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,0.0,0.0,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,11
4232,0.0,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,NaN,0.0,0.0,NaN,0.0,11
4233,0.0,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,0.0,11
4234,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,NaN,11


In [43]:
#We use LogisticRegression to test the performance of the filled_data by different ways

def LogisticRegression_model_test(recipes_nan,input_recipes_name):
    #split input data into 80%training data and 20%test data  
    X=recipes_nan.drop('cuisine',axis=1)
    y=recipes_nan['cuisine']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=5)
    clf_LogisticRegression = LogisticRegression(solver='sag',multi_class='multinomial')
    clf_LogisticRegression.fit(X_train, y_train)
    
    #Represent model's results for the test accuracy, precision, recall and f1 score
    y_pred_LogisticRegression_test=clf_LogisticRegression.predict(X_test)
    print(input_recipes_name,"'s Test Accuracy: %1.6f"%accuracy_score(y_test,y_pred_LogisticRegression_test))
    print(input_recipes_name,"'s Test F1: %1.6f\n" % f1_score(y_test, y_pred_LogisticRegression_test, average="weighted"))
    scores = cross_val_score(clf_LogisticRegression, X, y, scoring="neg_mean_squared_error", cv=5).mean()
    print(input_recipes_name,"'s MSE value is: ",(-1)*scores)
    
    #Represent model's results for the training accuracy, precision, recall and f1 score
    y_pred_LogisticRegression_train=clf_LogisticRegression.predict(X_train)
    print(input_recipes_name,"'s Train Accuracy: %1.6f"%accuracy_score(y_train,y_pred_LogisticRegression_train))
    print(input_recipes_name,"'s Train F1: %1.6f\n" % f1_score(y_train, y_pred_LogisticRegression_train, average="weighted"))

In [40]:
#RandomForest fill in missing value
from sklearn.ensemble import RandomForestRegressor

def RandomForest_fill(recipes_nan):
    
    '''
    Suppose a data with n features has missing values in feature T (a large number of missing values are more suitable). 
    T is regarded as a label, and the other n-1 features and the original data are regarded as a new feature matrix. 
    If the other features also have missing values, traverse all the features, starting with the one with the least missing values.

    To fill a feature, the missing value of other feature values is replaced by 0. 
    In this way, once each cycle, the feature with the missing value will be reduced by one


    Parameters
    ----------
    recipes_nan:    input data set with Nan

    X_train_RF:     feature T does not miss the value corresponding to n-1 feature + the original label
    Y_train_RF      feature T has no missing value
    X_test_RF:      feature T missing value corresponding to n-1 feature + original tag
    Y_test_RF:      feature T missing value (unknown)

    Ytest is the nan in feature T that we need to fill it by RandomForest
    '''
    #print("each column has Nan\n",recipes_nan.isnull().sum())
    recipes_nan_rank_keys=recipes_nan.isnull().sum().sort_values(ascending=False).keys()
    #print(recipes_nan_rank_keys)
    for i in range(len(recipes_nan_rank_keys)):
        Y_train_RF=recipes_nan[recipes_nan[recipes_nan_rank_keys[i]].notnull()][recipes_nan_rank_keys[i]]
        X_train_RF=recipes_nan[recipes_nan[recipes_nan_rank_keys[i]].notnull()].drop(recipes_nan_rank_keys[i],axis=1)
        X_test_RF=recipes_nan[recipes_nan[recipes_nan_rank_keys[i]].isnull()].drop(recipes_nan_rank_keys[i],axis=1)


        #the missing value of other feature values is replaced by 0
        X_train_RF[X_train_RF.isnull()]=0
        X_test_RF[X_test_RF.isnull()]=0
        #RandomForestRegressor training
        rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
        rfr.fit(X_train_RF,Y_train_RF)
        #using RF model to predict the data for feature T are Nan
        if X_test_RF.shape[0]>0:
            predict_filled_value = rfr.predict(X_test_RF)
            #recipes_nan[recipes_nan[recipes_nan_rank_keys[0]].isnull()][recipes_nan_rank_keys[0]]=predict_filled_value
            recipes_nan.loc[recipes_nan[recipes_nan_rank_keys[i]].isnull(),recipes_nan_rank_keys[i]]=predict_filled_value
    return(recipes_nan)

recipes_nan_RF=RandomForest_fill(recipes_nan)
LogisticRegression_model_test(recipes_nan_RF,'recipes_nan_RF')

recipes_nan_RF 's Test Accuracy: 0.767689
recipes_nan_RF 's Test F1: 0.767813

recipes_nan_RF 's MSE value is:  6.108234111514557
recipes_nan_RF 's Train Accuracy: 0.892857
recipes_nan_RF 's Train F1: 0.892750



In [30]:
#Filling the Nan by their mean value for 20% missing data set

from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")  
recipes_nan_Mean = imp_mean.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_Mean = pd.DataFrame(recipes_nan_Mean)
recipes_nan_Mean['cuisine']=recipes['cuisine']
recipes_nan_Mean.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_Mean,'recipes_nan_Mean')



recipes_nan_Mean 's Test Accuracy: 0.722877
recipes_nan_Mean 's Test F1: 0.722261

recipes_nan_Mean 's MSE value is:  7.18456372101312
recipes_nan_Mean 's Train Accuracy: 0.869244
recipes_nan_Mean 's Train F1: 0.869253



In [31]:
#Filling the Nan by 0 for 20% missing data set

imp_zero = SimpleImputer(missing_values=np.nan, strategy="constant",fill_value=0)  
recipes_nan_zero = imp_zero.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_zero = pd.DataFrame(recipes_nan_zero)
recipes_nan_zero['cuisine']=recipes['cuisine']
recipes_nan_zero.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_zero,'recipes_nan_zero')


recipes_nan_zero 's Test Accuracy: 0.728774
recipes_nan_zero 's Test F1: 0.728303

recipes_nan_zero 's MSE value is:  7.307472544608051
recipes_nan_zero 's Train Accuracy: 0.870425
recipes_nan_zero 's Train F1: 0.870353



In [39]:
#Filling the Nan by median for 20% missing data set

imp_median = SimpleImputer(missing_values=np.nan, strategy="median")  
recipes_nan_median = imp_median.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_median = pd.DataFrame(recipes_nan_median)
recipes_nan_median['cuisine']=recipes['cuisine']
recipes_nan_median.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_median,'recipes_nan_Mean')

recipes_nan_Mean 's Test Accuracy: 0.726415
recipes_nan_Mean 's Test F1: 0.725288

recipes_nan_Mean 's MSE value is:  7.44746664142033
recipes_nan_Mean 's Train Accuracy: 0.870720
recipes_nan_Mean 's Train F1: 0.870621



In [38]:
#Filling the Nan by most_frequent for 20% missing data set

imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy="most_frequent")  
recipes_nan_most_frequent = imp_most_frequent.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_most_frequent = pd.DataFrame(recipes_nan_most_frequent)
recipes_nan_most_frequent['cuisine']=recipes['cuisine']
recipes_nan_most_frequent.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_most_frequent,'recipes_nan_most_frequent')

recipes_nan_most_frequent 's Test Accuracy: 0.726415
recipes_nan_most_frequent 's Test F1: 0.725288

recipes_nan_most_frequent 's MSE value is:  7.44746664142033
recipes_nan_most_frequent 's Train Accuracy: 0.870720
recipes_nan_most_frequent 's Train F1: 0.870621



In [56]:
#Try 50 %missing  data set
recipes_nan=recipes_Nan_process(recipes,0.5)
recipes_nan

,'acorn squash',adobo,'african birdseye chile pepper',ale,'aleppo pepper','alfalfa sprouts','alfredo sauce',allspice,almond,'almond butter',...,yeast,'yellow curry paste','yellow food coloring','yellow split pea','yellow squash',yogurt,zaatar,zest,zucchini,cuisine
0,0.0,0.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0
1,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,...,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0
2,NaN,0.0,NaN,NaN,0.0,0.0,NaN,0.0,0.0,0.0,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0
3,0.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN,...,0.0,NaN,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0
4,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,...,NaN,0.0,0.0,NaN,0.0,0.0,NaN,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4231,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,0.0,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,NaN,0.0,11
4232,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,11
4233,NaN,NaN,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,11
4234,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,0.0,...,NaN,NaN,0.0,NaN,NaN,0.0,NaN,0.0,NaN,11


In [49]:
#Filling the Nan by their mean value for 50% missing data set

from sklearn.impute import SimpleImputer

imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")  
recipes_nan_Mean = imp_mean.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_Mean = pd.DataFrame(recipes_nan_Mean)
recipes_nan_Mean['cuisine']=recipes['cuisine']
recipes_nan_Mean.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_Mean,'recipes_nan_Mean')

recipes_nan_Mean 's Test Accuracy: 0.616745
recipes_nan_Mean 's Test F1: 0.618265

recipes_nan_Mean 's MSE value is:  9.61292825956205
recipes_nan_Mean 's Train Accuracy: 0.768005
recipes_nan_Mean 's Train F1: 0.768066



In [57]:
#Filling the Nan by 0 for 50% missing data set

imp_zero = SimpleImputer(missing_values=np.nan, strategy="constant",fill_value=0)  
recipes_nan_zero = imp_zero.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_zero = pd.DataFrame(recipes_nan_zero)
recipes_nan_zero['cuisine']=recipes['cuisine']
recipes_nan_zero.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_zero,'recipes_nan_zero')

recipes_nan_zero 's Test Accuracy: 0.607311
recipes_nan_zero 's Test F1: 0.609418

recipes_nan_zero 's MSE value is:  10.018716168051503
recipes_nan_zero 's Train Accuracy: 0.764758
recipes_nan_zero 's Train F1: 0.765130



In [58]:
#Filling the Nan by median for 50% missing data set

imp_median = SimpleImputer(missing_values=np.nan, strategy="median")  
recipes_nan_median = imp_median.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_median = pd.DataFrame(recipes_nan_median)
recipes_nan_median['cuisine']=recipes['cuisine']
recipes_nan_median.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_median,'recipes_nan_Median')

recipes_nan_Median 's Test Accuracy: 0.594340
recipes_nan_Median 's Test F1: 0.596812

recipes_nan_Median 's MSE value is:  10.186125002784523
recipes_nan_Median 's Train Accuracy: 0.762102
recipes_nan_Median 's Train F1: 0.762528



In [52]:
#Filling the Nan by most_frequent for 50% missing data set

imp_most_frequent = SimpleImputer(missing_values=np.nan, strategy="most_frequent")  
recipes_nan_most_frequent = imp_most_frequent.fit_transform(recipes_nan.drop('cuisine',axis=1))  
recipes_nan_most_frequent = pd.DataFrame(recipes_nan_most_frequent)
recipes_nan_most_frequent['cuisine']=recipes['cuisine']
recipes_nan_most_frequent.columns=recipes.columns

LogisticRegression_model_test(recipes_nan_most_frequent,'recipes_nan_most_frequent')

recipes_nan_most_frequent 's Test Accuracy: 0.594340
recipes_nan_most_frequent 's Test F1: 0.596593

recipes_nan_most_frequent 's MSE value is:  10.186125002784523
recipes_nan_most_frequent 's Train Accuracy: 0.762102
recipes_nan_most_frequent 's Train F1: 0.762528



In [53]:
recipes_nan_RF=RandomForest_fill(recipes_nan)
LogisticRegression_model_test(recipes_nan_RF,'recipes_nan_RF')

recipes_nan_RF 's Test Accuracy: 0.724057
recipes_nan_RF 's Test F1: 0.726207

recipes_nan_RF 's MSE value is:  5.321730135216413
recipes_nan_RF 's Train Accuracy: 0.848878
recipes_nan_RF 's Train F1: 0.849005

